In [1]:
from core.tasks import process_deep_entries_data
from core.helpers.common import (
    rm_punc_not_nums, rm_punc_not_nums_list,
    rm_stop_words_txt, rm_stop_words_txt_list,
    translate_to_english_txt,
    compose
)
from core.feature_selectors import UnigramFeatureSelector, BigramFeatureSelector
from core.classifiers.NaiveBayes_classifier import NaiveBayesClassifier
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
import random
from nltk.corpus import names, movie_reviews
import langid   

In [16]:
csv_file_path = '_playground/sample_data/nlp_out.csv'

print('PROCESSING DEEP ENTRIES DATA')
data = process_deep_entries_data(csv_file_path)
print('DONE')

print('REMOVING PUNC AND STOP WORDS')
stemmer = PorterStemmer()
rm_punc_and_stop = compose(
    rm_punc_not_nums_list,
    rm_stop_words_txt_list,
    lambda x: list(map(str.lower, x)),
    lambda x: list(map(stemmer.stem,x)) # comment this if we don't need stemming
)
#rm_punc_and_stop = lambda x: x
data = [(rm_punc_and_stop(str(ex).split()), l) for (ex, l) in data if langid.classify(str(ex))[0] == 'en']
print('DONE')

#data = [(list(movie_reviews.words(fileid)), category)
#       for category in movie_reviews.categories()
#      for fileid in movie_reviews.fileids(category)
#]
#print(data[0])
tags_data = {}
for ex, l in data:
    tags_data[l] = tags_data.get(l, '') + " "+ str(ex)
    
all_tokenized_documents = list(map(lambda x:x.split(), [v for k, v in tags_data.items()]))

print('SHUFFLING DATA')
random.shuffle(data)
print('DONE')

data_len = len(data)
test_len = int(data_len * 0.25)

print('TAKING OUT TEST/TRAIN DATA')
train_data = data[test_len:]
print("length of training data", len(train_data))
test_data = data[:test_len]
print('DONE')

print('COUNTING TAG FREQUENCIES in TRAIN DATA')
d = {}
for ex, l in train_data:
    d[l] = d.get(l, 0) + 1
print(d)
print('DONE')


PROCESSING DEEP ENTRIES DATA
DONE
REMOVING PUNC AND STOP WORDS
DONE
SHUFFLING DATA
DONE
TAKING OUT TEST/TRAIN DATA
length of training data 17718
DONE
COUNTING TAG FREQUENCIES in TRAIN DATA
{'WASH': 1434, 'Nutrition': 787, 'Cross': 691, 'Shelter': 1626, 'Health': 3359, 'Food': 3077, 'Education': 723, 'Agriculture': 604, 'Logistic': 364, 'Livelihood': 1126, 'NFI': 776, 'Protection': 3151}
DONE


In [17]:
print('CREATING FEATURE SELECTOR')
from core.tf_idf import relevant_terms
#most_relevant_terms = list(relevant_terms(all_tokenized_documents))
#selector = UnigramFeatureSelector.new(freq_words=most_relevant_terms)
selector = UnigramFeatureSelector.new(corpus=data, top=2000) # use top 2000 words
print('DONE')

# print('CREATING BIGRAM FEATURE SELECTOR')
# selector = BigramFeatureSelector.new(corpus=data, top=2000)
# selector = DocumentFeatureSelector.new(corpus=data, top=2000)
# print('DONE')

print('CREATING CLASSIFIER')
classifier = NaiveBayesClassifier.new(selector, rm_punc_and_stop, train_data)
print('DONE')

print('CALCULATING ACCURACY')
print(classifier.get_accuracy(test_data))

#print('CONFUSION MATRIX')
#print(classifier.get_confusion_matrix(test_data))

CREATING FEATURE SELECTOR
DONE
CREATING CLASSIFIER
DONE
CALCULATING ACCURACY
0.593972231628852


In [18]:
print(classifier.get_confusion_matrix(test_data))

            |   A                                             |
            |   g                   L               P         |
            |   r       E           i           N   r         |
            |   i       d           v   L       u   o         |
            |   c       u           e   o       t   t   S     |
            |   u       c       H   l   g       r   e   h     |
            |   l   C   a       e   i   i       i   c   e     |
            |   t   r   t   F   a   h   s       t   t   l   W |
            |   u   o   i   o   l   o   t   N   i   i   t   A |
            |   r   s   o   o   t   o   i   F   o   o   e   S |
            |   e   s   n   d   h   d   c   I   n   n   r   H |
------------+-------------------------------------------------+
Agriculture |<110>  5   2  18   6  29   4   2   .   1  19   2 |
      Cross |   2 <48>  7  12  20   5  17   7   8  37  38  14 |
  Education |   2   5<174>  1  11   2   6   2   3  21  25   6 |
       Food |  75  36  13<515> 81  62  2

In [20]:
from core.models import ClassifierModel
from django.conf import settings
settings.configure()
print(classifier)

ImproperlyConfigured: Requested setting DEFAULT_INDEX_TABLESPACE, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.